# BIDNAMIC CHALLENGE

1 - Import libraries, and create variables 

In [1]:
import psycopg2
import pandas as pd
hostname = 'localhost'
database = 'bidnamic'
username = 'postgres'
pwd = 'a'
port_id = 5432
conn = None

In [37]:
search_terms=pd.read_csv('search_terms.csv')
campaigns=pd.read_csv('campaigns.csv')
adgroups=pd.read_csv('adgroups.csv')


In [4]:
conn = psycopg2.connect(database="bidnamic",
                        user='postgres', password='a', 
                        host='127.0.0.1', port='5432'
)
conn.autocommit = True
cursor = conn.cursor()

2 - Create table in postgreSQL

In [39]:
tables = ["campaigns","adgroups","search_terms"]
columns= [campaigns.columns.tolist(),adgroups.columns.tolist(),
         search_terms.columns.tolist()]  
columnsType = [["NUMERIC(1000)","VARCHAR (1000) NOT NULL","VARCHAR (1000) NOT NULL"],
               ["NUMERIC(1000)","NUMERIC(1000)","VARCHAR (1000) NOT NULL","VARCHAR (1000) NOT NULL"],
               ["DATE","NUMERIC(1000)","NUMERIC(1000)","NUMERIC(1000)","FLOAT(10) NOT NULL",
                "FLOAT(10) NOT NULL","FLOAT(10) NOT NULL","VARCHAR (1000) NOT NULL"]]

for t,i in zip(tables,range(0,3)):
    sql = 'CREATE TABLE ' + t +' ('
    for a,b in zip(columns[i],columnsType[i]):
        sql += a+' '+b+','
    sql = sql [:-1]   
    sql += ');' 
    cursor.execute(sql)

3- Injecting the data CSV=>PostgreSQL with standard python implementation

In [40]:
for t,i in zip(tables,range(0,3)):
    sql = 'COPY ' + t +' ('
    for a,b in zip(columns[i],columnsType[i]):
        sql += a+','
    sql = sql [:-1]   
    sql += ') '
    sql += '''FROM '/Users/I/Desktop/bidnamic/'''
    sql += t +'''.csv'  CSV HEADER DELIMITER ',';'''
    #print(sql)
    cursor.execute(sql)

4- Showing the ROAS by country and priority

4.1- Innerjoin for ad_group_id
    - We can do it through standard python implementation, but it takes too much time
        sql = '''SELECT *
        FROM search_terms
        INNER JOIN adgroups
        ON search_terms.ad_group_id = adgroups.ad_group_id;'''
        cursor.execute(sql)
        for i in cursor.fetchall():
            print(i)
        sql = '''SELECT *
        FROM search_terms'''
        cursor.execute(sql)
    - We can do it through pandas python implementation, but it takes too much time
        innerJoin  = pd.read_sql_query('''SELECT *
        FROM search_terms
        INNER JOIN adgroups
        ON search_terms.ad_group_id = adgroups.ad_group_id''',con=conn)
    - We can query first through pandas python implementation, and then do it through pandas merge
        search_terms = pd.read_sql_query('select * from "search_terms"',con=conn)
        adgroups = pd.read_sql_query('select * from "adgroups"',con=conn)
        innerJoin = pd.merge(search_terms,adgroups,on='ad_group_id')
    This last solution is the fastest


In [46]:
search_terms = pd.read_sql_query('select * from "search_terms"',con=conn)
adgroups = pd.read_sql_query('select * from "adgroups"',con=conn)

In [41]:
innerJoin = pd.merge(search_terms,adgroups,on='ad_group_id')

4.2- Calculate ROAS

In [49]:
country = {}
priority = {}
for ijcv,ijc,ija in zip(innerJoin.conversion_value,innerJoin.cost,innerJoin.alias):
    ija = ija.split(" - ")        
    if ija[2] not in country:
        country[ija[2]] = [[],[]]
    if ija[2] not in priority:
        priority[ija[4]] =[[],[]]
    country[ija[2]][0] += [ijcv]
    country[ija[2]][1] += [ijc]
    priority[ija[4]][0] += [ijcv]
    priority[ija[4]][1] += [ijc]

In [52]:
for c in  country.keys():
    country[c][0] = sum(country[c][0])/len(country[c][0])
    country[c][1] = sum(country[c][1])/len(country[c][1])
    if country[c][1] != 0:
         country[c] = country[c][0]/country[c][1]
    else:
        country[c] = 0
for p in  priority.keys():
    priority[p][0] = sum(priority[p][0])/len(priority[p][0])
    priority[p][1] = sum(priority[p][1])/len(priority[p][1])
    if priority[p][1] !=0:
        priority[p] = priority[p][0]/priority[p][1]
    else:
        priority[p] = 0

5 - Showing ROAS

In [53]:
priority

{'LOW': 0, 'HIGH': 0, 'MEDIUM': 0}

In [54]:
country

{'GB': 4.0540028836590665}